In [215]:
import os
import glob
import re
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from netCDF4 import Dataset
from pvlib.solarposition import get_solarposition


In [5]:
os.chdir('/home/dhkim/projects/gk2aICT/')
from src.config import (
    GK2A_DATA_PATH,
)
from src.variables import (
    VAR2DSKEY,
    FD_VAR2FILE,
)
from src.tools.clip import (
    GK2AFDProcessor,
)


In [6]:
MODIS_PATH = '/mnt/sda1/research/data/etc/MCD12C1.A2019001.006.2020220162300.hdf'

In [3]:
def read_modis_land_cover(path,
                          name='Majority_Land_Cover_Type_1',
                          print_channels=True):

    f = SD(path, SDC.READ)

    if print_channels:
        data_dict = f.datasets()
        for idx, sds in enumerate(data_dict.keys()):
            print(idx, sds)

    sds_obj = f.select(name)
    data = sds_obj.get()

    return data


In [149]:
ko_range = {
    'ullatitude': 43.0,
    'ullongitude': 124.0,
    'lrlatitude': 33.0,
    'lrlongitude': 132.0,
}
ea_range = {
    'ullatitude': 53,
    'ullongitude': 77,
    'lrlatitude': 11,
    'lrlongitude': 150,
}
etc_range = {
    'ullatitude': 38.0,
    'ullongitude': 126.0,
    'lrlatitude': 37.0,
    'lrlongitude': 128.0,
}

In [12]:
base_path = GK2A_DATA_PATH

In [23]:
flist = glob.glob(os.path.join(base_path, '*swrad*'))
flist.sort()

In [161]:
swrad_vars = ['DSR', 'ASR', 'RSR']

In [195]:
target_range = ko_range

In [162]:
ii = 0
f = flist[ii]

In [163]:
obs_datetime = re.findall(string=f, pattern='[0-9]*.nc')[0][:-3]

In [201]:
ds = Dataset(os.path.join(base_path, f))
if ii == 0:
    gk2a = GK2AFDProcessor(gk2a_filename=cla_filename)
    i = np.arange(0, ds.dimensions['xdim'].size, dtype='f')
    j = np.arange(0, ds.dimensions['ydim'].size, dtype='f')
    i, j = np.meshgrid(i, j)
    (geos_lat, geos_lon) = gk2a.latlon_from_rowcol(j, i)

/home/dhkim/projects/gk2aICT/src/tools/clip.py:67: RuntimeWarning: invalid value encountered in sqrt
  Sd = np.sqrt( (42164.0*np.cos(x)*np.cos(y))**2 - (np.cos(y)**2 + 1.006739501*np.sin(y)**2)*1737122264)


In [202]:
cur_data = gk2a.cut_with_latlon(np.array(ds[cur_var][:]), **target_range)

In [203]:
cur_latlons = pd.DataFrame({
    'latitude': gk2a.cut_with_latlon(geos_lat, **target_range).flatten(),
    'longitude': gk2a.cut_with_latlon(geos_lon, **target_range).flatten(),
})

In [205]:
cur_latlons.shape

(128352, 2)

In [209]:
cur_latlons[:2].apply(
    lambda x: get_solarposition(obs_datetime, x.latitude, x.longitude)['zenith'],
    axis=1
)

,2021-11-18 05:50:00
0,70.193400
1,70.204118


In [ ]:
%%timeit -n 30
cur_latlons['zenith'] = cur_latlons[:2].apply(
    lambda x: get_solarposition(obs_datetime, x.latitude, x.longitude)['zenith'],
    axis=1
)

11.2 ms ± 262 µs per loop (mean ± std. dev. of 7 runs, 30 loops each)


In [189]:
get_solarposition(obs_datetime, cur_latlons.latitude[2], cur_latlons.longitude[2])

,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
2021-11-18 05:50:00,72.941345,72.995153,17.058655,17.004847,168.448455,14.861135


In [171]:
cur_latlons

,latitude,longitude
0,53.023765,76.929501
1,53.014949,77.005329
2,53.006170,77.080893
3,52.997429,77.156195
4,52.988724,77.231239
...,...,...
4144157,11.017644,149.912593
4144158,11.017942,149.933277
4144159,11.018241,149.953964
4144160,11.018540,149.974657


In [103]:
a.always_mask

True

In [25]:
a = ds[cur_var]

In [ ]:
a.obs

In [14]:
flist

['/mnt/sda1/research/data/tmp_gk2a/gk2a_ami_le2_swrad_fd020ge_202111180550.nc',
 '/mnt/sda1/research/data/tmp_gk2a/gk2a_ami_le2_swrad_fd020ge_202111180610.nc',
 '/mnt/sda1/research/data/tmp_gk2a/gk2a_ami_le2_swrad_fd020ge_202111180620.nc',
 '/mnt/sda1/research/data/tmp_gk2a/gk2a_ami_le2_swrad_fd020ge_202111180630.nc',
 '/mnt/sda1/research/data/tmp_gk2a/gk2a_ami_le2_swrad_fd020ge_202111180640.nc',
 '/mnt/sda1/research/data/tmp_gk2a/gk2a_ami_le2_swrad_fd020ge_202111180650.nc',
 '/mnt/sda1/research/data/tmp_gk2a/gk2a_ami_le2_swrad_fd020ge_202111180700.nc',
 '/mnt/sda1/research/data/tmp_gk2a/gk2a_ami_le2_swrad_fd020ge_202111180710.nc',
 '/mnt/sda1/research/data/tmp_gk2a/gk2a_ami_le2_swrad_fd020ge_202111180720.nc',
 '/mnt/sda1/research/data/tmp_gk2a/gk2a_ami_le2_swrad_fd020ge_202111180730.nc',
 '/mnt/sda1/research/data/tmp_gk2a/gk2a_ami_le2_swrad_fd020ge_202111180740.nc',
 '/mnt/sda1/research/data/tmp_gk2a/gk2a_ami_le2_swrad_fd020ge_202111180750.nc',
 '/mnt/sda1/research/data/tmp_gk2a/gk2a_

In [59]:
cmap = 'Reds'
# target_range = ko_range
target_datetime = '202111190450' # set your interesting datetime
# target_datetime = '202111190100'

In [60]:
cla_filename = f'gk2a_ami_le2_cla_fd020ge_{target_datetime}.nc'

In [61]:
ds = Dataset(os.path.join(base_path, cla_filename))
cl_proc = GK2AFDProcessor(gk2a_filename=cla_filename)
ca = np.array(ds['CA'][:])
cf = np.array(ds['CF'][:])


In [64]:
ds.names_of_product

'CT, CLL, CA, CF, CT_DQF, CF_DQF, CA_DQF'

In [68]:
a = ds['CT']

In [ ]:
a.